## Escenario pruebas 2 - Incluyendo nueva logica KM

In [ ]:

# --- Celda de Prueba para Etapa Final del Agente en Jupyter Notebook ---
import pandas as pd
import logging
import traceback
import json
from typing import Dict, List, Optional, Any
from langchain_core.messages import HumanMessage, BaseMessage, AIMessage

# --- 1. Importaciones Necesarias (Ajusta las rutas según tu proyecto) ---
# Clases de Estado y Enums
from graph.perfil.state import (
    EstadoAnalisisPerfil, PerfilUsuario, InfoPasajeros,
    FiltrosInferidos, EconomiaUsuario, InfoClimaUsuario
)
from utils.enums import (
    Transmision, NivelAventura, TipoUsoProfesional,
    DimensionProblematica, EstiloConduccion, TipoMecanica,
    FrecuenciaUso, DistanciaTrayecto, FrecuenciaViajesLargos # Asegurarse de importar los nuevos
)

test_config = {
    "configurable": {
        "thread_id": "test_thread_notebook_123"
    }
}
# Nodos del Grafo
# Asumimos que todos estos nodos existen y son importables
from graph.perfil.nodes import (
    # calcular_recomendacion_economia_modo1_node, # Asumimos que existe pero no lo usaremos directamente si el modo es 2
    calcular_flags_dinamicos_node,
    calcular_pesos_finales_node,
    formatear_tabla_resumen_node,
    calcular_km_anuales_postprocessing_node,
    buscar_coches_finales_node
)

# Configuración de LLMs
try:
    from config.llm import llm, llm_explicacion_coche
except ImportError:
    print("WARN: No se pudo importar la configuración de LLM. Las explicaciones podrían no funcionar.")
    llm_explicacion_coche = None

# Configurar logging básico para ver los DEBUG de los nodos
logging.basicConfig(level=logging.INFO)
logging.getLogger().setLevel(logging.INFO)


# --- Configuración de Pandas para Mejor Visualización ---
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 2000)
pd.set_option('display.max_colwidth', 150)

# --- 2. Simulación de un Estado Completo del Agente ---

# ✅ Elige qué perfil quieres probar cambiando esta variable
PERFIL_A_PROBAR = "PERFIL"  # Opciones: "FAMILIAR_PRACTICO", "GRAN_VIAJERO"

print(f"--- 🚗 CONFIGURANDO PRUEBA PARA PERFIL: {PERFIL_A_PROBAR} ---")

if PERFIL_A_PROBAR == "PERFIL":
    test_info_clima = InfoClimaUsuario(
        MUNICIPIO_ZBE=False, ZONA_LLUVIAS=False, ZONA_NIEBLAS=False, ZONA_NIEVE=False,
        ZONA_CLIMA_MONTA=False, ZONA_GLP=False, ZONA_GNV=False, cp_valido_encontrado=True
    )
    test_preferencias_usuario = PerfilUsuario(
        apasionado_motor="no", #activa premium y singular
        valora_estetica="no",  #activa singular
        prefiere_diseno_exclusivo="no", #activa singular
        circula_principalmente_ciudad = 'si',
        uso_profesional="no", 
        tipo_uso_profesional=None,
        altura_mayor_190="no", peso_mayor_100="no",
        coche_principal_hogar="si",
        rating_fiabilidad_durabilidad=9,
        rating_seguridad=5,
        rating_comodidad=5,
        rating_impacto_ambiental=9,
        rating_costes_uso=9, 
        rating_tecnologia_conectividad=5,
        transporta_carga_voluminosa="no", 
        necesita_espacio_objetos_especiales= None,
        arrastra_remolque="no", 
        tiene_garage="si",
        espacio_sobra_garage = "si",  
        problemas_aparcar_calle= None, # DimensionProblematica.LARGO/ANCHO/ALTO 
        tiene_punto_carga_propio="si",
        solo_electricos="no", 
        prioriza_baja_depreciacion="si",
        frecuencia_uso=FrecuenciaUso.DIARIO,
        distancia_trayecto=DistanciaTrayecto.MENOS_10_KM , # Esto ya dispara un alto kilometraje
        realiza_viajes_largos= 'no',
        frecuencia_viajes_largos=None,
        transmision_preferida=Transmision.AMBOS,
        estilo_conduccion=EstiloConduccion.TRANQUILO,
        aventura=NivelAventura.ninguna,
        
    )
    test_info_pasajeros = InfoPasajeros(suele_llevar_acompanantes= False ,
                                frecuencia= None, #Si suele_llevar_acompanantes= false: info_pasajeros.frecuencia a "nunca".["nunca", "ocasional", "frecuente"]
                                frecuencia_viaje_con_acompanantes = None, #ocasional , frecuente
                                num_ninos_silla=0, num_otros_pasajeros=0) 
    
    
    
    test_economia_usuario = EconomiaUsuario(modo=2, submodo=1, pago_contado=18000.0)


# 
initial_filtros_inferidos =  FiltrosInferidos( 
     #tipo_mecanica=[TipoMecanica.GASOLINA, TipoMecanica.DIESEL, TipoMecanica.GLP , TipoMecanica.GNV , TipoMecanica.HEVD , TipoMecanica.HEVG , TipoMecanica.PHEVG , TipoMecanica.PHEVD ] # Ejemplo
)

initial_state_dict: EstadoAnalisisPerfil = {
    "messages": [AIMessage(content="...")], 
    "info_clima_usuario": test_info_clima,
    "preferencias_usuario": test_preferencias_usuario, 
    "info_pasajeros": test_info_pasajeros,
    "economia": test_economia_usuario, 
    "filtros_inferidos": initial_filtros_inferidos,
    "pesos": None, "tabla_resumen_criterios": None, "coches_recomendados": None,
    "pregunta_pendiente": None, "penalizar_puertas_bajas": None, "priorizar_ancho": False,
    # Flags que se calcularán en calcular_flags_dinamicos_node
    "flag_penalizar_low_cost_comodidad": True, #Activando flags penalización comodidad flag penalizar depor comod y flag penalizar lowcost comod
    "flag_penalizar_deportividad_comodidad": True, #Activando flags penalización comodidad flag penalizar depor comod y flag penalizar lowcost comod
    "flag_penalizar_antiguo_por_tecnologia": True, # Lógica para Flag de Penalización por Antigüedad y Tecnología > 7
    "aplicar_logica_distintivo_ambiental": True,
    "es_municipio_zbe": False,
    "codigo_postal_extraido_temporal": None,
    "tipo_mensaje_cp_llm": None,
    "_decision_cp_validation": None,
    "penalizar_bev_reev_aventura_ocasional": False,
    "penalizar_phev_aventura_ocasional": False,
    "penalizar_electrificados_aventura_extrema": False,
    "favorecer_carroceria_montana": False,
    'favorecer_carroceria_comercial': False,
    'favorecer_carroceria_pasajeros_pro': False,
    'desfavorecer_carroceria_no_aventura': True, #Aventura 'ninguna' y no es clima de montaña. Activando flag para desfavorecer PICKUP/TODOTERRENO."
    'favorecer_suv_aventura_ocasional': False,
    "favorecer_pickup_todoterreno_aventura_extrema": False,
    "aplicar_logica_objetos_especiales": False,
    "favorecer_carroceria_confort": False,
    "flag_logica_uso_ocasional": False,
    "flag_favorecer_bev_uso_definido": False,
    "flag_penalizar_phev_uso_intensivo":False,
    "flag_favorecer_electrificados_por_punto_carga": True,
   
    # completar las flags con las que quiero hacer pruebas...
}


# --- 3. Ejecutar la Secuencia de Nodos de Finalización y Búsqueda ---
print("\n--- 🚀 INICIANDO SECUENCIA DE FINALIZACIÓN Y BÚSQUEDA ---")
current_state = initial_state_dict.copy()

try:
    # El orden aquí es importante para que los datos fluyan correctamente
    
    print("\n1. Calculando KM anuales...")
    output_km_anuales = calcular_km_anuales_postprocessing_node(current_state)
    current_state.update(output_km_anuales)
    print(f"✅ KM Anuales Estimados: {current_state.get('km_anuales_estimados')}")

    print("\n2. Calculando Flags dinámicos...")
    output_flags = calcular_flags_dinamicos_node(current_state)
    current_state.update(output_flags)

    print("\n3. Calculando Pesos finales...")
    output_pesos = calcular_pesos_finales_node(current_state)
    current_state.update(output_pesos)

    print("\n4. Generando Tabla Resumen de Criterios...")
    # Asumimos que tienes un nodo o función para esto
    # Si no, esta parte se puede comentar
    output_tabla = formatear_tabla_resumen_node(current_state)
    current_state.update(output_tabla)
    
    print("\n5. 🚀 Ejecutando Búsqueda Final en BigQuery...")
    output_final_busqueda = buscar_coches_finales_node(current_state , test_config)
    current_state.update(output_final_busqueda)
    
    print("\n\n--- 🏁 RESULTADO FINAL ---")
    print("\n--- Mensaje Final del Agente (Combinado) ---")
    if current_state.get('messages'):
        # El último mensaje debería ser el combinado de tabla + coches
        current_state['messages'][-1].pretty_print()
    # if current_state.get('coches_recomendados'):
    #     print(f"Se encontraron {len(current_state['coches_recomendados'])} coches:")
    #     df = pd.DataFrame(current_state['coches_recomendados'])
    #     # Seleccionar y reordenar columnas para una visualización más clara
    #     cols_display = [
    #         'score_total', 'nombre', 'precio_compra_contado', 'tipo_mecanica',
    #         'autonomia_uso_principal', 'tiempo_carga_min', 'potencia_maxima_carga_DC', 'km_ocasion'
    #     ]
    #     # Filtrar solo las columnas que existen en el DataFrame para evitar errores
    #     cols_existentes = [col for col in cols_display if col in df.columns]
    #     display(df[cols_existentes].round(2))
    else:
        print("No se encontraron coches que cumplan los criterios.")
        # Imprimir el mensaje de sugerencia que genera el nodo
        if current_state.get('messages'):
            current_state['messages'][-1].pretty_print()


except Exception as e_test:
    print(f"\n❌ ERROR durante la prueba de la secuencia de nodos: {e_test}")
    traceback.print_exc()

In [ ]:
import pandas as pd
coches_recomendados = current_state[('coches_recomendados')]

df_resultados = pd.DataFrame(coches_recomendados)
        
#  OPCIÓN 1: Imprimir el DataFrame directamente (Pandas usará las opciones de display)
# print("\n--- Vista DataFrame (Pandas Display Options) ---")
# display(df_resultados) # 'display()' suele dar mejor formato en notebooks que 'print(df)'

#OPCIÓN 2: Convertir a Markdown (como en tu nodo, pero para consola)
print("\n--- Vista Markdown ---")
columnas_deseadas = [ # Lista completa de columnas que seleccionas en BQ
            'nombre', 'ID', 'marca', 'modelo', 'score_total','cambio_automatico', 'tipo_mecanica', 
            'tipo_carroceria', 'indice_altura_interior', 'batalla', 'estetica', 
            'premium', 'singular', 'altura_libre_suelo', 'ancho', 'traccion', 
            'reductoras', 'puertas', 'plazas', 'precio_compra_contado',
            'fiabilidad', 'durabilidad', 'seguridad', 'comodidad', 'acceso_low_cost', 
            'deportividad', 'tecnologia', 'devaluacion', 'maletero_minimo', 
            'maletero_maximo', 'largo', 'autonomia_uso_maxima', 
            'distintivo_ambiental', 'anos_vehiculo', 'ocasion',
            'peso_original_kg', 'consumo_original',
        ]
columnas_existentes = [col for col in columnas_deseadas if col in df_resultados.columns]
if columnas_existentes:
            if 'score_total' in df_resultados.columns: # Formatear score para legibilidad
                df_resultados['score_total'] = df_resultados['score_total'].apply(lambda x: f"{x:.4f}" if isinstance(x, float) else x)
            print(df_resultados[columnas_existentes].to_markdown(index=False))
else:
            print("WARN: No se encontraron columnas esperadas para mostrar en formato tabla.")
            print("Resultados crudos (lista de dicts):", coches_recomendados)

In [ ]:
print(current_state.keys())

In [ ]:
print(current_state['codigo_postal_usuario'])
print('-----------------------')
print(current_state['info_clima_usuario'])
print('-----------------------')
print(current_state['preferencias_usuario'])
print('-----------------------')
print(current_state['filtros_inferidos'])
print('-----------------------')
print(current_state['info_pasajeros'])
print('-----------------------')
print(current_state['economia'])
print('-----------------------')
print(current_state['pesos'])
print('-----------------------')
print("info_pasajeros:",current_state['info_pasajeros'])
print('-----------------------')

In [ ]:
print("flag_penalizar_deportividad_comodidad:", current_state['flag_penalizar_deportividad_comodidad'])
print('-----------------------')
print("flag_penalizar_low_cost_comodidad:", current_state['flag_penalizar_low_cost_comodidad'])
print('-----------------------')
print("penalizar_puertas_bajas:" , current_state['penalizar_puertas_bajas'])
print('-----------------------')
print("flag_penalizar_antiguo_por_tecnologia:" , current_state['flag_penalizar_antiguo_por_tecnologia'])
print('-----------------------')
print("aplicar_logica_distintivo_ambiental:" ,  current_state['aplicar_logica_distintivo_ambiental'])
print('-----------------------')
print("es municipio_zbe: " , current_state['es_municipio_zbe'])
print('-----------------------')
print("penalizar_bev_reev_aventura_ocasional: " , current_state['penalizar_bev_reev_aventura_ocasional'])
print('-----------------------')
print("penalizar_phev_aventura_ocasional: " , current_state['penalizar_phev_aventura_ocasional'])
print('-----------------------')
print("penalizar_electrificados_aventura_extrema: " , current_state['penalizar_electrificados_aventura_extrema'])
print('-----------------------')
print('favorecer_carroceria_montana', current_state['favorecer_carroceria_montana'])
print('-----------------------')
print('favorecer_carroceria_comercial', current_state['favorecer_carroceria_comercial'])
print('-----------------------')
print('favorecer_carroceria_pasajeros_pro:', current_state['favorecer_carroceria_pasajeros_pro'])
print('-----------------------')
print('desfavorecer_carroceria_no_aventura:', current_state['desfavorecer_carroceria_no_aventura']) #Aventura nula y no en montaña desfavorece PICKUP/TODOTERRENO
print('-----------------------')
print('favorecer_suv_aventura_ocasional:', current_state['favorecer_suv_aventura_ocasional']) #OK
print('-----------------------')
print('favorecer_pickup_todoterreno_aventura_extrema:', current_state['favorecer_pickup_todoterreno_aventura_extrema'])
print('-----------------------')
print('aplicar_logica_objetos_especiales:', current_state['aplicar_logica_objetos_especiales'])#OK 
print('-----------------------')
print('favorecer_carroceria_confort:', current_state['favorecer_carroceria_confort']) #OK
print('-----------------------')
print('flag_logica_uso_ocasional:' , current_state['flag_logica_uso_ocasional'])
print('-----------------------')
print('flag_favorecer_bev_uso_definido:' , current_state['flag_favorecer_bev_uso_definido'])
print('-----------------------')
print('flag_penalizar_phev_uso_intensivo:' , current_state['flag_penalizar_phev_uso_intensivo'])
print('-----------------------')
print('flag_favorecer_electrificados_por_punto_carga:' , current_state['flag_favorecer_electrificados_por_punto_carga'])
print('-----------------------')
print('km_anuales_estimados:' , current_state['km_anuales_estimados'])
